In [1]:
import numpy as np
from numpy import distutils
import distutils
import pandas as pd
import pickle
from methods import read_pickle
from methods import find_index_v2
from sklearn.svm import SVR
from numpy import distutils
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.ensemble import VotingRegressor
import plotly.graph_objects as go
import keras

data_cleanned_v3 = read_pickle(path_name = 'data\data_processed_experiment.pickle')

## Is RM a useful input?
### To conduct this problem, we use the same regressor with building prediction, i.e., RF to do two experiments
### The considered weather features for the two experiments are the same, while the only difference is the input of the second experiment is RM benchmarks with long historical data, which is normally considered as less effective than one month of data, and the relative differences as we presented in the report and a conference paper. 
### Result shows RM based prediction improves the MASE by 0.11 on average (around 16%) 

In [2]:
def split_training_dataframe(dict_df, training_type='Solar0', training_length=24*4*31, prediction_length=24*4*31):
    
    df = dict_df[training_type].copy()
    df['YEAR'] = df.index.year
    df['MONTH'] = df.index.month
    df['DATE'] = df.index.day
    df['HOUR'] = df.index.hour
    df['TIME'] = df.index.time
    df['WEEKDAY'] = df.index.weekday
    df['ISWEEKEND'] = np.where((df['WEEKDAY'] == 5) | (df['WEEKDAY'] == 6), True, False)
    if training_type.find('Building') != -1:
        df['LAG_DATA'] = df['consumption'].shift(24*4*31)
    df_training = df[-training_length - prediction_length: -prediction_length]
    
    # This if is to predict unseen data. Ask Nam personally if you are interested
    if prediction_length == 0:
        prediction_length = training_length
        
    df_prediction = df[-prediction_length:]
    
    return df_training, df_prediction, df
def get_training_parameter(df_training, training_type):    
    # Training data
    dew_tempC_tr = df_training['dewpoint_temperature (degC)']
    tempC_tr = df_training['temperature (degC)']
    radiation_tr = df_training['surface_solar_radiation (W/m^2)']
    thermal_radiation_tr = df_training['surface_thermal_radiation (W/m^2)']
    cloudcover_tr = df_training['total_cloud_cover (0-1)']
    humidity_tr = df_training['relative_humidity ((0-1))']
    wind_speed_tr = df_training['wind_speed (m/s)']
    hour_tr = df_training['HOUR'].astype('category').cat.codes.astype(float)
    weekday_tr = df_training['WEEKDAY'].astype('category').cat.codes.astype(float)
    time_tr = df_training['TIME'].astype('category').cat.codes.astype(float)   
    if training_type.find('Building') != -1:
        y_training = np.array(df_training['consumption'])
        lag_data_tr = df_training['LAG_DATA']
        occup_tr = df_training['occupancy (0-1)']
        x_training_RF = np.column_stack((occup_tr, tempC_tr, humidity_tr, cloudcover_tr, radiation_tr, lag_data_tr, time_tr, weekday_tr))    
    if training_type.find('Solar') != -1:
        y_training = np.array(df_training["demand(kWh)/production(kW)"])
        x_training_RF = np.column_stack((cloudcover_tr, radiation_tr, thermal_radiation_tr, humidity_tr, wind_speed_tr, time_tr))    
    return x_training_RF, y_training


def get_Nov_parameter(df, training_type, prediction_length, occu = False):    
    # Predicting data  
    length = prediction_length
    weather = df[training_type][-length:]
    dew_tempC_oct = weather['dewpoint_temperature (degC)']
    tempC_oct = weather['temperature (degC)']
    radiation_oct = weather['surface_solar_radiation (W/m^2)']
    thermal_radiation_pr = weather['surface_thermal_radiation (W/m^2)']
    cloudcover_oct = weather['total_cloud_cover (0-1)']
    humidity_oct = weather['relative_humidity ((0-1))']
    wind_speed_oct = weather['wind_speed (m/s)']
    # hour_pr = df_prediction['HOUR'].astype('category').cat.codes.astype(float)
    weekday_oct = pd.CategoricalIndex(weather.index.weekday).codes.astype(float)
    time_oct = pd.CategoricalIndex(weather.index.time).codes.astype(float)
    ## Summer time switch
    # time_oct[252:] = (time_oct[252:]+4)%96
    
    if training_type.find('Building') != -1:
        lag_data_oct = df[training_type]['consumption'].values[-length:]
        occup_oct = df['occupancy'][:length]
        x_prediction_RF = np.column_stack((occup_oct, tempC_oct, humidity_oct, cloudcover_oct, radiation_oct, lag_data_oct, time_oct, weekday_oct)) if occu else np.column_stack((tempC_oct, humidity_oct, cloudcover_oct, radiation_oct, lag_data_oct, time_oct, weekday_oct))
    
    if training_type.find('Solar') != -1:
        lag_data_oct = df[training_type]["demand(kWh)/production(kW)"].values[-length:]
        x_prediction_RF = np.column_stack((cloudcover_oct, radiation_oct, thermal_radiation_pr, humidity_oct, wind_speed_oct, time_oct))   
    return x_prediction_RF
def GB_training(x_training_RF, y_training):
    reg1 = RandomForestRegressor(n_estimators=700,
                               min_samples_split=5, random_state=96,
                               n_jobs=-1)
    reg2 = BaggingRegressor(n_estimators=700)
    gb = reg1
    # gb = VotingRegressor(estimators=[('fg', reg1), ('bg', reg2)])
    gb.fit(x_training_RF, y_training)
    return gb

def spot_prediction(dict_df, training_type='Solar0'):
    
    training_length = 24*4*30 
    prediction_length = 24*4*30
    
    df_training, df_prediction, df_solar = split_training_dataframe(dict_df, training_type, training_length, prediction_length)
    x_training_RF, y_training = get_training_parameter(df_training, training_type)
    # RF_regression = RF_training(x_training_RF, y_training)
    RF_regression = GB_training(x_training_RF, y_training)
    nov_parameter = get_Nov_parameter(dict_df, training_type, prediction_length, occu = True)
    y_predict_nov = RF_regression.predict(nov_parameter)
    # This if is to predict unseen data. Ask Nam personally if you are interested
    # if prediction_length == 0:
    #     prediction_length = training_length
    # y_naive = df_solar[training_type].shift(2688)[-prediction_length:].values if training_type.find('Solar') != -1 else df_solar['consumption'].shift(2688)[-prediction_length:].values
    # return y_predict_RF, y_naive, y_real, y_predict_nov 
    return y_predict_nov

In [3]:
from numba import jit

@jit
def weighted_DTW(series_1, series_2, weight):
    #### Series_1 and Series_2 are time series list, weight is weighted matrix list with two dimention
    #### default weight can be np.ones(len(series_1))
    #### weight stands for different attentions on the sub-patterns
    l1 = len(series_1)
    l2 = len(series_2)
    cum_sum = np.full((l1 + 1, l2 + 1), np.inf)
    cum_sum[0, 0] = 0.
    for i in range(l1):
        for j in range(l2):
            diff = (series_1[i] - series_2[j])
            distance = diff*diff*weight[i][j]
            cum_sum[i + 1, j + 1] = distance
            cum_sum[i + 1, j + 1] += min(cum_sum[i, j + 1],
                                            cum_sum[i + 1, j],
                                            cum_sum[i, j])
    acc_cost_mat = cum_sum[1:, 1:]
    return np.sqrt(acc_cost_mat[-1][-1])
class RM_model:
    """A class for representative daily data discovery"""

    ### fixed threshold
    def motif_dtw_24_w(self, time_series, m, threshold, weight):
        weighted_matrix = np.ones((len(weight), len(weight)))
        for i in range(len(weight)):
            for j in range(len(weight)):
                weighted_matrix[i][j] = max(weight[i], weight[j])
        # find the motif with most sub-patterns near than threshold
        length = len(time_series)-m
        sliding = int(length/m)
        distance_profile = []
        average_profile = []
        motif_co = 0
        time_series = np.array(time_series)
        for i in range(sliding+1):
            pattern = time_series[i*m:i*m+m]
            counter = 0
            average_profile.append(0)
            for j in range(sliding+1):
                distance = weighted_DTW(pattern, time_series[j*m:j*m+m], weighted_matrix)
                average_profile[i] = average_profile[i]+distance
                if distance < threshold:
                    counter+=1
            average_profile[i] = average_profile[i]/sliding
            distance_profile.append(counter - average_profile[i]*0.001)
        motif_co = np.argsort(distance_profile)[-1]
        motif_se = np.argsort(distance_profile)[-2]
        motif = dict (
            distance_profile = distance_profile,
            motif_position = motif_co*m,
            motif_sec = motif_se*m,
            average_profile = average_profile
        )

    ### Adoptable threshold
    def motif_dtw_24_w_dy(self, time_series, m, weight):
        weighted_matrix = np.ones((len(weight), len(weight)))
        for i in range(len(weight)):
            for j in range(len(weight)):
                weighted_matrix[i][j] = max(weight[i], weight[j])
        # find the motif with most sub-patterns near than threshold
        length = len(time_series)-m
        sliding = int(length/m)
        distance_stack = []
        distance_profile = []
        average_profile = []
        motif_co = 0
        # change to array for numba accelerataion
        time_series = np.array(time_series)
        for i in range(sliding+1):
            pattern = time_series[i*m:i*m+m]
            average_profile.append(0)
            for j in range(sliding+1):
                distance = weighted_DTW(pattern, time_series[j*m:j*m+m], weighted_matrix)
                average_profile[i] = average_profile[i]+distance
                distance_stack.append(distance)
            average_profile[i] = average_profile[i]/sliding
        threshold = np.median(distance_stack)
        # print('Distance matrix: ',distance_stack)
        for k in range(sliding+1):  
            pattern = time_series[i*m:i*m+m]  
            counter = 0
            for l in range(sliding+1):
                distance = distance_stack[k*(sliding+1)+l]
                if distance < threshold:
                    counter+=1
            distance_profile.append(counter - average_profile[k]*0.001)
        motif_co = np.argsort(distance_profile)[-1]
        motif_se = np.argsort(distance_profile)[-2]
        motif = dict (
            distance_profile = distance_profile,
            motif_position = motif_co*m,
            motif_sec = motif_se*m,
            average_profile = average_profile
        )
        return motif

    def __init__(self, data_cleanned):
         self.data_cleanned = data_cleanned
    
    def motifs_discovery_v3(self, user_type, motifs_range):
        ## Inputs chosen by Granger causality
        motif_data =  self.data_cleanned[user_type][motifs_range[0]:] if motifs_range[1] ==0 else self.data_cleanned[user_type][motifs_range[0]:motifs_range[1]]
        targeted_series = motif_data["demand(kWh)/production(kW)"] if user_type.find('Solar') != -1 else motif_data['consumption']
        ## No special attention applied
        motifs = self.motif_dtw_24_w_dy(targeted_series, 96, np.ones((96)))
        motif_pattern = targeted_series[motifs['motif_position']:motifs['motif_position']+96]
        # print('Refined Motif at: ',motif_data.index[motifs['motif_position']])
        temperature_motif = motif_data['temperature (degC)'][motifs['motif_position']:motifs['motif_position']+96]
        cloudcover_motif = motif_data ['total_cloud_cover (0-1)'][motifs['motif_position']:motifs['motif_position']+96]
        humidity_motif = motif_data ['relative_humidity ((0-1))'][motifs['motif_position']:motifs['motif_position']+96]
        radiation_motif = motif_data ['surface_solar_radiation (W/m^2)'][motifs['motif_position']:motifs['motif_position']+96]
        sealevel_motif = motif_data ['mean_sea_level_pressure (Pa)'][motifs['motif_position']:motifs['motif_position']+96]
        wind_motif = motif_data ['wind_speed (m/s)'][motifs['motif_position']:motifs['motif_position']+96]
        times_motif = pd.CategoricalIndex(motif_data.index.time).codes.astype(float) 
        return motifs, motif_data, motif_pattern, temperature_motif, cloudcover_motif, humidity_motif, radiation_motif, sealevel_motif, wind_motif, times_motif

In [4]:

def RM_train_test_formation(data_cleanned, user_type, test_days):
    train_days = int(len(data_cleanned[user_type])/96-test_days)
    train_range = [-24*4*(train_days+test_days),-24*4*test_days] 
    motifs_range = train_range
    motifs_range[0] = motifs_range[1]-30*96  ### remove this line to test long range motifs (takes longer)
    test_range = [-24*4*test_days, 0]
    RM = RM_model(data_cleanned)
    motifs, motif_data, motif_pattern, temperature_motif, cloudcover_motif, humidity_motif, radiation_motif, sealevel_motif, wind_motif, times_motif = RM.motifs_discovery_v3(user_type, motifs_range)
    train_range = [-24*4*(train_days+test_days),-24*4*test_days] 
    train_data = data_cleanned[user_type][train_range[0]:].copy() if train_range[1] ==0 else data_cleanned[user_type][train_range[0]:train_range[1]].copy()
    y_train = train_data["demand(kWh)/production(kW)"].values.reshape(train_days, 96) if user_type.find('Solar') != -1 else train_data['consumption'].values.reshape(train_days, 96)
    cloudcover_train = train_data ['total_cloud_cover (0-1)'].values.reshape(train_days, 96)
    humidity_train = train_data ['relative_humidity ((0-1))'].values.reshape(train_days, 96)
    radiation_train = train_data ['surface_solar_radiation (W/m^2)'].values.reshape(train_days, 96)
    temperature_train = train_data['temperature (degC)'].values.reshape(train_days, 96)
    sealevel_train = train_data['mean_sea_level_pressure (Pa)'].values.reshape(train_days, 96)
    wind_train = train_data['wind_speed (m/s)'].values.reshape(train_days, 96)
    temperature_train = [i-temperature_motif.values for i in temperature_train] 
    sealevel_train = [i-sealevel_motif.values for i in sealevel_train]
    wind_train = [i-wind_motif.values for i in wind_train]
    cloudcover_train = [i-cloudcover_motif.values for i in cloudcover_train] 
    humidity_train = [i-humidity_motif.values for i in humidity_train]
    radiation_train = [i-radiation_motif.values for i in radiation_train]
    times_train = pd.CategoricalIndex(train_data.index.time).codes.astype(float)
    sun_cycle_train = np.maximum(np.sin(2*np.pi *(times_train+16)/96), 0 ).reshape(train_days, 96)
    times_train = times_train.reshape(train_days, 96)
    weekday_train = pd.CategoricalIndex(train_data.index.weekday).codes.astype(float).reshape(train_days, 96)
    motifs_train = [motif_pattern for i in range(train_days)]
    motifs_test = [motif_pattern for i in range(test_days)]
    motifs_train = np.array(motifs_train)
    motifs_test = np.array(motifs_test)
    month_train = pd.CategoricalIndex(train_data.index.month).codes.astype(float).reshape(train_days, 96)
    # year_cycle_train = 1+np.sin(2*np.pi *(month_train-8)/12)
    year_cycle_train = abs(np.sin(2*np.pi*(month_train-5)/24))
    year_cycle_train = year_cycle_train.reshape(train_days, 96)
    weekend_train = np.where((weekday_train == 5) | (weekday_train == 6), True, False)
    ### Test data
    test_data = data_cleanned[user_type][test_range[0]:].copy()
    y_test = test_data['demand(kWh)/production(kW)'].values.reshape(test_days, 96) if user_type.find('Solar') != -1 else test_data['consumption'].values.reshape(test_days, 96)
    y_max_test = [[np.max(i), np.min(i)] for i in y_test]
    y_max_test = np.array(y_max_test)
    cloudcover_test = test_data ['total_cloud_cover (0-1)'].values.reshape(test_days, 96)
    humidity_test = test_data ['relative_humidity ((0-1))'].values.reshape(test_days, 96)
    radiation_test = test_data ['surface_solar_radiation (W/m^2)'].values.reshape(test_days, 96)
    temperature_test = test_data['temperature (degC)'].values.reshape(test_days, 96)
    sealevel_test = test_data['mean_sea_level_pressure (Pa)'].values.reshape(test_days, 96)
    wind_test = test_data['wind_speed (m/s)'].values.reshape(test_days, 96)
    temperature_test = [i-temperature_motif.values for i in temperature_test] 
    cloudcover_test = [i-cloudcover_motif.values for i in cloudcover_test] 
    humidity_test = [i-humidity_motif.values for i in humidity_test]
    radiation_test = [i-radiation_motif.values for i in radiation_test]
    sealevel_test = [i-sealevel_motif.values for i in sealevel_test]
    wind_test = [i-wind_motif.values for i in wind_test]
    times_test = pd.CategoricalIndex(test_data.index.time).codes.astype(float)
    sun_cycle_test = np.maximum(np.sin(2*np.pi *(times_test+16)/96), 0 ).reshape(test_days, 96)
    times_test = times_test.reshape(test_days, 96)
    weekday_test = pd.CategoricalIndex(test_data.index.weekday).codes.astype(float).reshape(test_days, 96)
    month_test = pd.CategoricalIndex(test_data.index.month).codes.astype(float).reshape(test_days, 96)
    month_test = month_test+month_train[-1][-1]+1
    year_cycle_test = abs(np.sin(2*np.pi*(month_test-5)/24))
    year_cycle_test = year_cycle_test.reshape(test_days, 96)
    weekend_test = np.where((weekday_test == 5) | (weekday_test == 6), True, False)
    if user_type.find('Solar0') != -1:
        ## Solar 0 has data less than a year hence the yearly cycle needs to be adjusted 
        year_cycle_train = abs(np.sin(2*np.pi*(month_train-1)/24))
        year_cycle_test = abs(np.sin(2*np.pi*(month_test-1)/24))
        x_train_cnn = np.dstack((motifs_train, temperature_train , cloudcover_train, radiation_train, year_cycle_train))
        x_test_cnn = np.dstack((motifs_test,  temperature_test, cloudcover_test,  radiation_test, year_cycle_test))
    else:
        # x_train_cnn = np.dstack((motifs_train, temperature_train , cloudcover_train, radiation_train, year_cycle_train))
        x_train_cnn = np.dstack((motifs_train, humidity_train, temperature_train , cloudcover_train,wind_train,  radiation_train, year_cycle_train))
        x_test = np.column_stack((motifs_test, cloudcover_test, radiation_test, times_test, month_test)) 
        # x_test_cnn = np.dstack((motifs_test, temperature_test, cloudcover_test, radiation_test, year_cycle_test)) 
        x_test_cnn = np.dstack((motifs_test, humidity_test, temperature_test, cloudcover_test, wind_test, radiation_test, year_cycle_test))
    return x_train_cnn, y_train, x_test_cnn, y_test, train_days

In [7]:
data_cleanned = data_cleanned_v3.copy()
## Solar 3 has abnormal data in 2019, so we replace that period with Solar 2 data (discussed in report)
data_cleanned['Solar3'][:96*380] =  data_cleanned_v3['Solar2'][:96*380]
## Similarly for Solar 5 and Solar 0, but we don't have data can fill the abnormals, so we just remove them
data_cleanned['Solar5'] =  data_cleanned_v3['Solar5'][96*300:]
data_cleanned['Solar0'] =  data_cleanned_v3['Solar0'][96*120:]
user_type = 'Solar5'
test_days = 30
improvements = 0
solar_prediction = []
for user_type in ['Solar'+str(i) for i in np.arange(6)]:
    x_train_cnn, y_train, x_test_cnn, y_test, train_days = RM_train_test_formation(data_cleanned, user_type, test_days)
    x_train_rf = x_train_cnn.reshape(train_days*96, x_train_cnn.shape[-1])
    x_test_rf = x_test_cnn.reshape(test_days*96, x_test_cnn.shape[-1])
    RF_regression = GB_training(x_train_rf, y_train.reshape(-1))
    RF_regression_vanilla = spot_prediction(data_cleanned, user_type)
    prediction_length = 30*96
    y_naive = data_cleanned[user_type]['demand(kWh)/production(kW)'].shift(30*96)[-prediction_length:].values
    y_real = y_test.reshape(-1)
    y_predict_RF = RF_regression.predict(x_test_rf)
    solar_prediction.append(y_predict_RF)
    sust = np.mean(np.abs(y_naive - y_real))
    diff_RF = np.mean(np.abs(y_real - y_predict_RF))
    diff_RF_vanilla = np.mean(np.abs(y_real - RF_regression_vanilla))
    print('Forecasting for: ', user_type)
    print('RF with RM MASE: ', diff_RF/sust)
    print('RF without RM MASE: ', diff_RF_vanilla/sust)
    improvements += diff_RF_vanilla/sust - diff_RF/sust
print('Average improvements on MASE: ', improvements/5)

Forecasting for:  Solar0
RF with RM MASE:  0.7113814733836814
RF without RM MASE:  0.6949972070696021
Forecasting for:  Solar1
RF with RM MASE:  0.5733170157986679
RF without RM MASE:  0.6892263059310786
Forecasting for:  Solar2
RF with RM MASE:  0.5931009629332507
RF without RM MASE:  0.7014221847909562
Forecasting for:  Solar3
RF with RM MASE:  0.5937702816196088
RF without RM MASE:  0.7059561432911288
Forecasting for:  Solar4
RF with RM MASE:  0.5513387927294428
RF without RM MASE:  0.6935199899759179
Forecasting for:  Solar5
RF with RM MASE:  0.5905373596390703
RF without RM MASE:  0.6873095286178302
Average improvements on MASE:  0.11179709471455841


In [61]:
import plotly.graph_objects as go

In [351]:
user_type = 'Solar0'
data_shown = data_cleanned[user_type]
Time = data_shown.index
m=96
RM_pos = motifs['motif_position']
m=48
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x= Time, y=data_shown['demand(kWh)/production(kW)'], line_color= 'green', line_width = 0.8, name = 'synthetic net demand', opacity = 0.7
    )
)
# fig.add_vrect(
#         x0=Time[RM_pos], x1=Time[RM_pos+m-1],
#         fillcolor="LightSalmon", opacity=0.5,
#         layer="below", line_width=0,
#         annotation_text="RM", annotation_position="top left",
# ),
# fig.add_vrect(
#         x0=Time[-prediction_length], x1=Time[-1],
#         fillcolor="yellow", opacity=0.5,
#         layer="below", line_width=0,
#         annotation_text="Predict range", annotation_position="top left",   
# ),

fig.layout.title = 'Check '+user_type+' data'
fig.layout.xaxis.title = 'Time'
# fig.layout.title = "MAPE: "+str(MAPE(sample,real_sample))
fig.layout.yaxis.title = "Electricity generation (kWh)"
fig.layout.plot_bgcolor='rgba(0,0,0,0)'
fig.layout.font.family="Times New Roman" ## "Times New Roman Black"
fig.layout.font.size = 20
fig.layout.width = 1000
fig.layout.height = 500
# fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,showgrid=False )
# fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
fig.show()

## Is ResNet useful?
### To conduct the second question, we use the same RM based inputs, but this time we used ResNet instead of RF
### Since ResNet is computational expensive, we applied very simple block layers instead of Res50 or Res101. The test patience is also very low as this is a show case for proving ResNet has the potential for reaching better result than RF. In real world with high performance computing, this method might leverage grokking and each user's model can be customised to reach the best result (as what we did in competition phase 1) 

In [6]:
from methods import NN_model
# Solar_pre_model = NN_model(x_train_cnn, y_train, 96) 
# model_cnn = Solar_pre_model.ResNet(error_matric= 'mae', block_layers = [2, 2, 2, 2])

In [371]:
for user_type in ['Solar'+str(i) for i in np.arange(5)]:
    x_train_cnn, y_train, x_test_cnn, y_test, train_days = RM_train_test_formation(data_cleanned, user_type, test_days)
    Solar_pre_model = NN_model(x_train_cnn, y_train, 96) 
    model_cnn = Solar_pre_model.ResNet(error_matric= 'mae', block_layers = [6, 8, 2, 2])
    test_epochs = 2000
    test_patience = 20
    # y_predict_max = model_cnn_max.predict(x_test_cnn).reshape(test_days,2)
    model_cnn.fit(x_train_cnn, y_train, epochs = test_epochs, validation_data = (x_test_cnn, y_test), batch_size=24, verbose = 0, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=test_patience)])
    y_predict_cnn = model_cnn.predict(x_test_cnn).reshape(test_days*96)
    y_real = y_test.reshape(-1)
    y_naive = data_cleanned[user_type]['demand(kWh)/production(kW)'].shift(30*96)[-prediction_length:].values
    sust = np.mean(np.abs(y_naive - y_real))
    diff_NN = np.mean(np.abs(y_real - y_predict_cnn))
    print('ResNet with RM MASE for '+ user_type +': ', diff_NN/sust)

Refined Motif at:  2020-10-13 00:00:00
ResNet with RM MASE for Solar0:  0.6843127072266586
Refined Motif at:  2020-10-13 00:00:00
ResNet with RM MASE for Solar1:  0.6725440774884773
Refined Motif at:  2020-10-13 00:00:00
ResNet with RM MASE for Solar2:  0.6457855617946378
Refined Motif at:  2020-10-13 00:00:00
ResNet with RM MASE for Solar3:  0.6577857328831389
Refined Motif at:  2020-10-13 00:00:00
ResNet with RM MASE for Solar4:  0.5716898665802677
